In [ ]:
import pandas as pd

from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

import base64

In [ ]:


newglobalChartDataMonthly = pd.read_csv('mysite/data/newglobalChartDataMonthly.csv')
featureHistory = pd.read_csv('mysite/data/feature-history.csv')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


# comment JupyterDash,  uncomment Dash for Python Anywhere deployment

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
#app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server

spotify_params = ["danceability",	"energy",	"key", "loudness", "mode", "speechiness", "acousticness",	"instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature"]
regions = ["Asia","North America", "South America", "Europe", "Oceania"]

colorscales = px.colors.named_colorscales()

In [ ]:
app.layout = html.Div([
    html.Div([
    html.Div([

        html.H1("World map"),
        html.P("Audio feature"),
        dcc.Dropdown(id='var_choice'  , value='danceability'  , options=[{'label': i, 'value': i} for i in  spotify_params]),
        html.P("Color Scale"),
        dcc.Dropdown(
            id='colorscale',
            options=[{"value": x, "label": x}
                     for x in colorscales],
            value='viridis'
        ),
        dcc.Graph(id='my-graph'),

    ], className="eight columns"),
    html.Div([

        html.H1("Feature Durchschnitt"),
        html.P("Audio feature"),
        dcc.Dropdown(id='var_choice2'  , value='danceability'  , options=[{'label': i, 'value': i} for i in  spotify_params]),
        html.P("Region"),
        dcc.Dropdown(
            id='region',
            options=[{"value": x, "label": x}
                     for x in regions],
            value='Europe'
        ),
        dcc.Graph(id='my-graph2'),

        html.H1("Feature Trend"),
        dcc.Dropdown(id='var_choice3'  , value='danceability'  , options=[{'label': i, 'value': i} for i in  spotify_params]),
        html.Img(id='my-img1'),
        #html.Img(src='data:image/png;base64,{}'.format(encoded_image)),

    ], className="three columns"),
#, style={'width': '30%', 'display': 'inline-block'}
], className="row")



])#, style={'columnCount': 2})

@app.callback(     Output('my-graph'        , 'figure'    ) ,
                  [Input("colorscale", "value"), Input('var_choice'       , 'value'     )] )
def update_figure(scale, var_choice):


    fig = px.choropleth(newglobalChartDataMonthly, locations="country",
                    locationmode="country names",
                    color=newglobalChartDataMonthly[var_choice],
                    animation_frame=newglobalChartDataMonthly["date"],
color_continuous_scale=scale,
                 width=1400, height=800)

    return fig

@app.callback(     Output('my-graph2'        , 'figure'    ) ,
                  [Input("region", "value"), Input('var_choice2'       , 'value'     )] )
def update_figure(region, var_choice2):


    data = featureHistory[(featureHistory['feature'] == var_choice2) & (featureHistory['region'] == region)]
    fig = px.bar(data, x='year', y='value')


    return fig

@app.callback(     Output('my-img1'        , 'src'    ) ,
                  [Input('var_choice3'       , 'value'     )] )
def update_figure(var_choice3):


    image_filename = 'mysite/data/RegionPng/' + var_choice3 + '.png'
    encoded_image = base64.b64encode(open(image_filename, 'rb').read())
    return 'data:image/png;base64,{}'.format(encoded_image.decode())

In [ ]:
# comment for Python Anywhere deployyment
app.run_server(port=8050)